In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
load_dotenv()
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
api_key = os.getenv('GROQ_API_KEY')

In [3]:
llm = ChatGroq(model="llama-3.1-70b-versatile",
               api_key=api_key,
               temperature=0.2)

In [4]:
from langchain.utilities import SQLDatabase

In [5]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [6]:
from langchain_experimental.sql import SQLDatabaseChain

In [7]:
db_chain = SQLDatabaseChain.from_llm(llm, db,verbose=True,return_direct=True)

In [8]:
db_chain('How many t-shirts do we have left for nike in extra small size and white color')

C:\Users\User\AppData\Local\Temp\ipykernel_7480\3085544697.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  db_chain('How many t-shirts do we have left for nike in extra small size and white color')




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color
SQLQuery:Question: How many t-shirts do we have left for nike in extra small size and white color
SQLQuery: 
SELECT `stock_quantity` 
FROM `t_shirts` 
WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White'
SQLResult: [(43,)]
> Finished chain.


{'query': 'How many t-shirts do we have left for nike in extra small size and white color',
 'result': '[(43,)]'}

In [9]:
db_chain.invoke("SELECT SUM(stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size='XS')")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size='XS')
SQLQuery:Question: SELECT SUM(stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size='XS')
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS' LIMIT 5;
SQLResult: [(Decimal('43'),)]
> Finished chain.


{'query': "SELECT SUM(stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size='XS')",
 'result': "[(Decimal('43'),)]"}

In [10]:
db_chain.invoke('How much is the price of the inventory for all small size t-shirts')



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts
SQLQuery:Question: How much is the price of the inventory for all small size t-shirts
SQLQuery: 
SELECT SUM(`price` * `stock_quantity`) AS `total_price` 
FROM `t_shirts` 
WHERE `size` = 'S'
SQLResult: [(Decimal('18018'),)]
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts',
 'result': "[(Decimal('18018'),)]"}

In [11]:
db_chain.invoke("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size ='S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size ='S'
SQLQuery:Question: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size ='S'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S' LIMIT 5;
SQLResult: [(Decimal('18018'),)]
> Finished chain.


{'query': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size ='S'",
 'result': "[(Decimal('18018'),)]"}

In [12]:
db_chain.invoke("If I have to sell all the Levi's T-shirts today with discounts applied. How much revenue does our store generate (post discounts) ")



> Entering new SQLDatabaseChain chain...
If I have to sell all the Levi's T-shirts today with discounts applied. How much revenue does our store generate (post discounts) 
SQLQuery:Question: If I have to sell all the Levi's T-shirts today with discounts applied. How much revenue does our store generate (post discounts) 

SQLQuery: 
```sql
SELECT 
    SUM(`price` * (1 - `pct_discount` / 100) * `stock_quantity`) AS total_revenue
FROM 
    t_shirts
JOIN 
    discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id
WHERE 
    t_shirts.brand = 'Levi';
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT \n    SUM(`price` * (1 - `pct_discount` / 100) * `stock_quantity`) ' at line 1")
[SQL: ```sql
SELECT 
    SUM(`price` * (1 - `pct_discount` / 100) * `stock_quantity`) AS total_revenue
FROM 
    t_shirts
JOIN 
    discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id
WHERE 
    t_shirts.brand = 'Levi';
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [13]:
db_chain.invoke("""SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue
FROM (SELECT SUM(price*stock_quantity) as total_amount, t_shirt_id  
FROM t_shirts
WHERE brand = 'Levi'
GROUP BY t_shirt_id)
a
LEFT JOIN 
discounts
ON
a.t_shirt_id = discounts.t_shirt_id""")



> Entering new SQLDatabaseChain chain...
SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue
FROM (SELECT SUM(price*stock_quantity) as total_amount, t_shirt_id  
FROM t_shirts
WHERE brand = 'Levi'
GROUP BY t_shirt_id)
a
LEFT JOIN 
discounts
ON
a.t_shirt_id = discounts.t_shirt_id
SQLQuery:Question: What is the total revenue for Levi brand t-shirts after applying discounts?
SQLQuery: 
SELECT SUM(a.`total_amount` * ((100-COALESCE(`discounts`.`pct_discount`,0))/100)) as `total_revenue`
FROM (SELECT SUM(`t_shirts`.`price`*`t_shirts`.`stock_quantity`) as `total_amount`, `t_shirts`.`t_shirt_id`  
FROM `t_shirts`
WHERE `t_shirts`.`brand` = 'Levi'
GROUP BY `t_shirts`.`t_shirt_id`)
a
LEFT JOIN 
`discounts`
ON
a.`t_shirt_id` = `discounts`.`t_shirt_id`
SQLResult: [(Decimal('27713.800000'),)]
> Finished chain.


{'query': "SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue\nFROM (SELECT SUM(price*stock_quantity) as total_amount, t_shirt_id  \nFROM t_shirts\nWHERE brand = 'Levi'\nGROUP BY t_shirt_id)\na\nLEFT JOIN \ndiscounts\nON\na.t_shirt_id = discounts.t_shirt_id",
 'result': "[(Decimal('27713.800000'),)]"}

In [14]:
db_chain.invoke("""SELECT SUM(price * stock_quantity)
                        FROM t_shirts
                        WHERE brand = 'Levi' """)



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity)
                        FROM t_shirts
                        WHERE brand = 'Levi' 
SQLQuery:Question: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM t_shirts WHERE `brand` = 'Levi'
SQLResult: [(Decimal('31326'),)]
> Finished chain.


{'query': "SELECT SUM(price * stock_quantity)\n                        FROM t_shirts\n                        WHERE brand = 'Levi' ",
 'result': "[(Decimal('31326'),)]"}

In [15]:
db_chain.invoke("How many white color Levi's t shirts do we have available")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts do we have available
SQLQuery:Question: How many white color Levi's t shirts do we have available
SQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(38,), (12,), (83,)]
> Finished chain.


{'query': "How many white color Levi's t shirts do we have available",
 'result': '[(38,), (12,), (83,)]'}

In [16]:
db_chain.invoke("""SELECT SUM(stock_quantity)
FROM t_shirts
WHERE brand = 'Levi' AND color = 'White'""")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity)
FROM t_shirts
WHERE brand = 'Levi' AND color = 'White'
SQLQuery:Question: SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 5
SQLResult: [(Decimal('133'),)]
> Finished chain.


{'query': "SELECT SUM(stock_quantity)\nFROM t_shirts\nWHERE brand = 'Levi' AND color = 'White'",
 'result': "[(Decimal('133'),)]"}

In [17]:
few_shots = [
    {
        "Question": "How many t-shirts do we have left for nike in extra small size and white color",
        "SQLQuery": "SELECT SUM(stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size='XS')",
        "SQLResult": "Result of the SQL query",
        "Answer": '43'
    },
    {
        "Question": 'How much is the price of the inventory for all small size t-shirts',
        "SQLQuery": "SELECT SUM(price * stock_quantity FROM t_shirts WHERE size ='S'",
        "SQLResult": "Result of the SQL query",
        "Answer": '18018'
    },
    {
        "Question": "If I have to sell all the Levi's T-shirts today with discounts applied. How much revenue does our store generate (post discounts)",
        "SQLQuery": """SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue
                        FROM (SELECT SUM(price*stock_quantity) as total_amount, t_shirt_id  
                        FROM t_shirts
                        WHERE brand = 'Levi'
                        GROUP BY t_shirt_id)
                        a
                        LEFT JOIN 
                        discounts
                        ON
                        a.t_shirt_id = discounts.t_shirt_id""",
        "SQLResult": "Result of the SQL query",
        "Answer": '27713.800000'
    },
    {
        "Question": "If we have to sell all the Levi's T-shirts today. How much revenue will our store make",
        "SQLQuery": """SELECT SUM(price * stock_quantity)
                        FROM t_shirts
                        WHERE brand = 'Levi' """,
        "SQLResult": "Result of the SQL query",
        "Answer": '31326'
    },
    {
        "Question": "How many white color Levi's t shirts do we have available",
        "SQLQuery": """SELECT SUM(stock_quantity)
                        FROM t_shirts
                        WHERE brand = 'Levi' AND color = 'White'""",
        "SQLResult": "Result of the SQL query",
        "Answer": '133'
    },
]

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\User\AppData\Local\Temp\ipykernel_7480\3701169180.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\User\miniconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [21]:
from langchain.vectorstores import Chroma

In [22]:
vectorstore = Chroma.from_texts(
    texts = to_vectorize,
    embedding=embeddings,
    metadatas=few_shots
)

In [23]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

example_selector.select_examples({"Question": "How many Adidas T shirts do I have left in my store?"})

[{'Answer': '43',
  'Question': 'How many t-shirts do we have left for nike in extra small size and white color',
  'SQLQuery': "SELECT SUM(stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size='XS')",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '133',
  'Question': "How many white color Levi's t shirts do we have available",
  'SQLQuery': "SELECT SUM(stock_quantity)\n                        FROM t_shirts\n                        WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

In [24]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [25]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [26]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question","SQLQuery","SQLResult","Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [87]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt + """
\nReturn the result of the query execution. Do not include the SQL query or any explanations. You can add little preamble text to communicate the answer properly
""",
    suffix=PROMPT_SUFFIX + """
\nReturn only the numerical result of the query execution. Do not include the SQL query or any explanations. You can add little preamble text to communicate the answer properly
""",
    input_variables=["input","table_info","top_k"]
)

In [90]:
new_chain = SQLDatabaseChain.from_llm(
                llm=llm,
                db=db,
                prompt=few_shot_prompt,
                verbose=True
                )

In [91]:
new_chain.invoke('What brand of shirts will I sell to make the most money and how much will it be')



> Entering new SQLDatabaseChain chain...
What brand of shirts will I sell to make the most money and how much will it be
SQLQuery:To determine the brand of shirts that will generate the most revenue and the amount of revenue, we need to calculate the total revenue for each brand.

SQLQuery: SELECT `brand`, SUM(`price` * `stock_quantity`) as total_revenue
FROM t_shirts
GROUP BY `brand`
ORDER BY total_revenue DESC
LIMIT 1
SQLResult: [('Levi', Decimal('31326'))]
Answer:The brand of shirts that will generate the most revenue is Levi, and the total revenue will be 31326.
> Finished chain.


{'query': 'What brand of shirts will I sell to make the most money and how much will it be',
 'result': 'The brand of shirts that will generate the most revenue is Levi, and the total revenue will be 31326.'}

In [72]:
a =new_chain.invoke("If I have to sell all the white Nike T-shirts today with discounts applied. How much revenue does our store generate (post discounts)")



> Entering new SQLDatabaseChain chain...
If I have to sell all the white Nike T-shirts today with discounts applied. How much revenue does our store generate (post discounts)
SQLQuery:Question: If I have to sell all the white Nike T-shirts today with discounts applied. How much revenue does our store generate (post discounts)
SQLQuery: SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue
                        FROM (SELECT SUM(price*stock_quantity) as total_amount, t_shirt_id  
                        FROM t_shirts
                        WHERE brand = 'Nike' AND color = 'White'
                        GROUP BY t_shirt_id)
                        a
                        LEFT JOIN 
                        discounts
                        ON
                        a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('3244.000000'),)]
> Finished chain.


In [73]:
a

{'query': 'If I have to sell all the white Nike T-shirts today with discounts applied. How much revenue does our store generate (post discounts)',
 'result': "[(Decimal('3244.000000'),)]"}

In [61]:
# Strip unwanted parts
result_string = a['result']
cleaned_string = result_string.split("'")[1]  # Extract '3244.000000'
numeric_value = float(cleaned_string)

print(numeric_value)  # Output: 3244.0


3244.0


In [ ]:
ast.li